In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pymongo
import requests
import pandas as pd
import time

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.mars_db
collection = db.items

In [3]:
# initialize dictionary to store scraped information
mars_dictionary = {}

# mars news website
url = 'https://mars.nasa.gov/news/?page=0&per_page=15&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

# use 'requests' to retrieve requests object and then convert to html through beautifulsoup
response = requests.get(url)
soup = bs(response.text,'lxml')

# print(soup.prettify())

# scrape using beautifulsoup
first_article = soup.find('div',class_='slide')
news_title = first_article.find('div',class_='content_title').a.text.strip()
news_summary = first_article.find('div',class_='rollover_description_inner').text.strip()

# store info into dictionary
mars_dictionary['news_summary'] = news_summary
mars_dictionary['news_title'] = news_title

In [6]:
# start up chromebrowser to scrape images from JPL's website
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path)

images_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
jpl_url = 'https://www.jpl.nasa.gov'

browser.visit(images_url)

# assign current page's html to 'html' variable
html = browser.html

# use beautifulsoup to parse the html
soup = bs(html,'html.parser')

# click on the 'Full Image' link from featured image
browser.click_link_by_id('full_image')
time.sleep(1)
# assign html from current browser
image_html = browser.html
image_soup = bs(image_html,'html.parser')

# scrape for featured image's url
try:
    modal_img_url = image_soup.find('div',class_='fancybox-inner').img['src']
    # build full url 
    featured_image_url = jpl_url + modal_img_url

    # store info in dictionary
    mars_dictionary['featured_image_url'] = featured_image_url
except:
    time.sleep(5)
    modal_img_url = image_soup.find('div',class_='fancybox-inner').img['src']
    # build full url 
    featured_image_url = jpl_url + modal_img_url

    # store info in dictionary
    mars_dictionary['featured_image_url'] = featured_image_url

In [7]:
modal_img_url

'/spaceimages/images/mediumsize/PIA17761_ip.jpg'

In [8]:
import tweepy

# Twitter API Keys
consumer_key = "Ed4RNulN1lp7AbOooHa9STCoU"
consumer_secret = "P7cUJlmJZq0VaCY0Jg7COliwQqzK0qYEyUF9Y0idx4ujb3ZlW5"
access_token = "839621358724198402-dzdOsx2WWHrSuBwyNUiqSEnTivHozAZ"
access_token_secret = "dCZ80uNRbFDjxdU2EckmNiSckdoATach6Q8zb7YYYE5ER"


# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [9]:
target_user = 'MarsWxReport'

mars_weather_tweets = api.user_timeline(target_user)

for tweet in mars_weather_tweets:
    if 'Sol' in tweet['text']:
        mars_weather = tweet['text']

# store into dictionary
mars_dictionary['mars_weather_tweet'] = mars_weather

In [110]:
# initialize dictionary
mars_facts = {}
mars_facts_url = 'https://space-facts.com/mars/'

# request html from url and import into beautifulsoup
response = requests.get(mars_facts_url)
soup = bs(response.text,'lxml')

# find table -> tbody
table_html = str(soup.table).replace("\n","")
marsfacts_df = pd.read_html(table_html,index_col=0)[0]
marsfacts_df.index.names = ['Description']
marsfacts_df.columns = ['Values']
marsfacts_html_string = marsfacts_df.to_html(header=False).replace("\n","")

In [111]:
marsfacts_html_string

'<table border="1" class="dataframe">  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

In [296]:
# start up chromebrowser to scrape hemisphere images 
hem_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

browser.visit(hem_url)

hem_html = browser.html
hem_soup = bs(hem_html,'html.parser')

products_html = hem_soup.find_all('div',class_='item')

hemisphere_image_urls = []
usgs_url = 'https://astrogeology.usgs.gov'

for product in products_html:
    title = product.h3.text
    
    browser.click_link_by_partial_text(title)
    
    hem2_html = browser.html
    hem2_soup = bs(hem2_html,'html.parser')
    
    temp_url = hem2_soup.find('img',class_='wide-image')['src']
    
    temp_dictionary = {}
    
    temp_dictionary['img_url'] = usgs_url + temp_url
    temp_dictionary['title'] = title
    
    hemisphere_image_urls.append(temp_dictionary)
    
    browser.visit(hem_url)

In [297]:
hemisphere_image_urls

[{'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [305]:
mars_dictionary['facts_html'] = marsfacts_html_string
mars_dictionary['hemispheres'] = hemisphere_image_urls

In [306]:
mars_dictionary